# 1. Importing and installing relevant packages for our project

In [ ]:
pip install pyradiomics

In [ ]:
pip install -U segmentation-models

In [ ]:
import os 
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import PIL
import time
from IPython import display
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dropout, Conv2D, MaxPooling2D, concatenate, Reshape, Conv2DTranspose, LeakyReLU, BatchNormalization, Activation, UpSampling2D 
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import models 
from tensorflow.keras import backend as K
from keras.callbacks import History 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.layers import Flatten, Dense
import seaborn
from keras.optimizers import RMSprop
from sklearn.feature_selection import RFECV
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
sm.set_framework('tf.keras')

In [ ]:
from __future__ import print_function
import SimpleITK as sitk
import six
from radiomics import featureextractor, getTestCase, glcm, glrlm, glszm, imageoperations, shape, getImageTypes, getFeatureClasses, getParameterValidationFiles

# 2. Assigning a seed

In [ ]:
seed = 1
#random.seed = seed
#np.random.seed = seed

# 3. Defining variables

In [ ]:
SIZE = 256
CHANNELS = 1
CHANNELS_3C = 3

# 4. Importing datasets

Directories

In [ ]:
supression_images_dir = os.path.join("../input/xray-bone-shadow-supression/augmented/augmented/source/")
supression_targets_dir = os.path.join("../input/xray-bone-shadow-supression/augmented/augmented/target/")
supression_images_list = os.listdir(supression_images_dir)
supression_target_list = os.listdir(supression_targets_dir)

image_path = os.path.join("../input/chest-xray-masks-and-labels/Lung Segmentation/CXR_png")
mask_path = os.path.join("../input/chest-xray-masks-and-labels/Lung Segmentation/","masks/")
images = os.listdir(image_path)
mask = os.listdir(mask_path)

covid_images_dir_train = os.path.join("../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/")
covid_images_dir_test = os.path.join("../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/")
covid_images_metadata = "../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv"
covid_images_summary = "../input/coronahack-chest-xraydataset/Chest_xray_Corona_dataset_Summary.csv"
covid_images_train = os.listdir(covid_images_dir_train)
covid_images_test = os.listdir(covid_images_dir_test)

Catching images

*Supression*

In [ ]:
supression_images = list()
supression_targets = list()

In [ ]:
j = 0
for i in tqdm(supression_target_list):
    if j < 4000:
        path_images = supression_images_dir + str(i)
        path_targets = supression_targets_dir + str(i)
        image = cv2.imread(path_images,cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
        image = cv2.bitwise_not(image)
        image = cv2.equalizeHist(image)
        targets = cv2.imread(path_targets,cv2.IMREAD_GRAYSCALE) 
        targets = cv2.resize(targets, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
        targets = cv2.bitwise_not(targets)
        targets = cv2.equalizeHist(targets)
        supression_images.append(image)
        supression_targets.append(targets)
    j += 1

In [ ]:
import gc
#del supression_images
#del image
#del supression_targets
#del targets
gc.collect()

In [ ]:
SUPRESSION_IMAGES = np.array(supression_images)
SUPRESSION_IMAGES = SUPRESSION_IMAGES.reshape((len(SUPRESSION_IMAGES), SIZE, SIZE, 1))
SUPRESSION_IMAGES = SUPRESSION_IMAGES.astype('float32') / 255

SUPRESSION_TARGETS = np.array(supression_targets)
SUPRESSION_TARGETS = SUPRESSION_TARGETS.reshape((len(SUPRESSION_TARGETS), SIZE, SIZE, 1))
SUPRESSION_TARGETS = SUPRESSION_TARGETS.astype('float32') / 255

In [ ]:
SUPRESSION_IMAGES_3C = list()
SUPRESSION_TARGETS_3C = list()

for img in SUPRESSION_IMAGES:
    img_3C = np.concatenate((img,)*3, axis=-1)
    SUPRESSION_IMAGES_3C.append(img_3C)
    
for img in SUPRESSION_TARGETS:
    img_3C = np.concatenate((img,)*3, axis=-1)
    SUPRESSION_TARGETS_3C.append(img_3C)
    
SUPRESSION_IMAGES_3C = np.array(SUPRESSION_IMAGES_3C)
SUPRESSION_TARGETS_3C = np.array(SUPRESSION_TARGETS_3C)

In [ ]:
print(SUPRESSION_IMAGES_3C.shape,SUPRESSION_TARGETS_3C.shape)

*Segmentation*

In [ ]:
mask = [fName.split(".png")[0] for fName in mask]
image_file_name = [fName.split("_mask")[0] for fName in mask]
check = [i for i in mask if "mask" in i]

testing_files = set(os.listdir(image_path)) & set(os.listdir(mask_path))
training_files = check

ima = list()
mas = list()
ima_canny = list()
ima_otsu = list()
j = 0
for i in tqdm(testing_files):
    im = cv2.imread(os.path.join(image_path,i),cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    im = cv2.equalizeHist(im)
    im2 = cv2.GaussianBlur(im,(5,5),0)
    im3 = cv2.Canny(im2,56,56)
    ret,im2 = cv2.threshold(im2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    im2 = cv2.bitwise_not(im2)
    mask = cv2.imread(os.path.join(mask_path,i),cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    _,mask = cv2.threshold(mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ima.append(im)
    mas.append(mask)
    ima_otsu.append(im2)
    ima_canny.append(im3)
    
for i in tqdm(training_files): 
    im = cv2.imread(os.path.join(image_path,i.split("_mask")[0]+".png"),cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    im = cv2.equalizeHist(im)
    im2 = cv2.GaussianBlur(im,(5,5),0)
    im3 = cv2.Canny(im2,56,56)
    ret,im2 = cv2.threshold(im2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    im2 = cv2.bitwise_not(im2)
    mask = cv2.imread(os.path.join(mask_path,i+".png"),cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    _,mask = cv2.threshold(mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ima.append(im)
    mas.append(mask)
    ima_otsu.append(im2)
    ima_canny.append(im3)

In [ ]:
SEGMENTATION_IMAGES = np.array(ima)
SEGMENTATION_MASKS = np.array(mas)

SEGMENTATION_IMAGES = SEGMENTATION_IMAGES.reshape((len(SEGMENTATION_IMAGES), SIZE, SIZE, 1))
SEGMENTATION_MASKS = SEGMENTATION_MASKS.reshape((len(SEGMENTATION_MASKS), SIZE, SIZE, 1))

SEGMENTATION_IMAGES = SEGMENTATION_IMAGES.astype('float32') / 255
SEGMENTATION_MASKS = SEGMENTATION_MASKS.astype('float32') / 255

In [ ]:
SEGMENTATION_IMAGES_3C = list()
SEGMENTATION_MASKS_3C = list()

for img in SEGMENTATION_IMAGES:
    img_3C = np.concatenate((img,)*3, axis=-1)
    SEGMENTATION_IMAGES_3C.append(img_3C)
    
for img in SEGMENTATION_MASKS:
    img_3C = np.concatenate((img,)*3, axis=-1)
    SEGMENTATION_MASKS_3C.append(img_3C)
    
SEGMENTATION_IMAGES_3C = np.array(SEGMENTATION_IMAGES_3C)
SEGMENTATION_MASKS_3C = np.array(SEGMENTATION_MASKS_3C)

In [ ]:
names = list()
files_tes = list(testing_files)
files_tra = list(training_files)
for name in files_tes:
    names.append(name)
for name in files_tra:
    names.append(name)
names

In [ ]:
seg = list()
for item in names:
    nm = ''.join([str(elem) for elem in item.split(".png", 1)])
    seg.append(nm)
seg

*COVID-19*

In [ ]:
metadata = pd.read_csv("../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv")

metadata_type_train = metadata[metadata['Dataset_type']=='TRAIN']
metadata_name_train = metadata_type_train['X_ray_image_name']
metadata_name_train = np.array(metadata_name_train)

metadata_type_test = metadata[metadata['Dataset_type']=='TEST']
metadata_name_test = metadata_type_test['X_ray_image_name']
metadata_name_test = np.array(metadata_name_test)

In [ ]:
covid_images = list()
covid_images_otsu = list()
covid_images_canny = list()
covid_image_name = list()

for i in tqdm(metadata_name_train):
    path_images_train = covid_images_dir_train + str(i)
    image = cv2.imread(path_images_train,cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    image2 = cv2.GaussianBlur(image,(5,5),0)
    image3 = cv2.Canny(image2,56,56)
    ret,image2 = cv2.threshold(image2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image2 = cv2.bitwise_not(image2)
    covid_images.append(image)
    covid_images_otsu.append(image2)
    covid_images_canny.append(image3)
    covid_image_name.append(i)
    
for i in tqdm(metadata_name_test):    
    path_images_test = covid_images_dir_test + str(i)
    image = cv2.imread(path_images_test,cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    image2 = cv2.GaussianBlur(image,(5,5),0)
    image3 = cv2.Canny(image2,56,56)
    ret,image2 = cv2.threshold(image2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image2 = cv2.bitwise_not(image2)
    covid_images.append(image)
    covid_images_otsu.append(image2)
    covid_images_canny.append(image3)
    covid_image_name.append(i)

In [ ]:
COVID_IMAGES = np.array(covid_images)

COVID_IMAGES = COVID_IMAGES.reshape((len(COVID_IMAGES), SIZE, SIZE, 1))

COVID_IMAGES = COVID_IMAGES.astype('float32') / 255

In [ ]:
COVID_IMAGES_3C = list()

for img in COVID_IMAGES:
    img_3C = np.concatenate((img,)*3, axis=-1)
    COVID_IMAGES_3C.append(img_3C)
    
COVID_IMAGES_3C = np.array(COVID_IMAGES_3C)

# 5 - Defining the metrics for the models

In [ ]:
def iou_score(y_pred, y_true, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true, -1) + K.sum(y_pred, -1) - intersection
    iou = (intersection + smooth)/(union + smooth)
    return iou

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# 6. Bone supression from Chest X-Ray

In [ ]:
n_classes = 1
activation = 'sigmoid'
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
BATCH_SIZE = 16
X_train, X_test, y_train, y_test = train_test_split(SUPRESSION_IMAGES, SUPRESSION_TARGETS, test_size=0.30, random_state=12)
#X_train_3C, X_test_3C, y_train_3C, y_test_3C = train_test_split(SUPRESSION_IMAGES_3C, SUPRESSION_TARGETS_3C, test_size=0.30, random_state=12)

In [ ]:
input_shape=(SIZE,SIZE,1)
classes=1
kernel_size = 3
filter_depth = (64,128,256,512,0)
    
img_input = Input(shape=input_shape)

conv1 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(img_input)
batch1 = BatchNormalization()(conv1)
act1 = Activation("relu")(batch1)
pool1 = MaxPooling2D(pool_size=(2, 2))(act1)

conv2 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(pool1)
batch2 = BatchNormalization()(conv2)
act2 = Activation("relu")(batch2)
pool2 = MaxPooling2D(pool_size=(2, 2))(act2)

conv3 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool2)
batch3 = BatchNormalization()(conv3)
act3 = Activation("relu")(batch3)
pool3 = MaxPooling2D(pool_size=(2, 2))(act3)

conv4 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool3)
batch4 = BatchNormalization()(conv4)
act4 = Activation("relu")(batch4)

conv5 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act4)
batch5 = BatchNormalization()(conv5)
act5 = Activation("relu")(batch5)

up6 = UpSampling2D(size=(2, 2))(act5)
conv6 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up6)
batch6 = BatchNormalization()(conv6)
act6 = Activation("relu")(batch6)
concat6 = concatenate([act3,act6])

up7 = UpSampling2D(size=(2, 2))(concat6)
conv7 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up7)
batch7 = BatchNormalization()(conv7)
act7 = Activation("relu")(batch7)
concat7 = concatenate([act2,act7])

conv8 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(concat7)
batch8 = BatchNormalization()(conv8)
act8 = Activation("relu")(batch8)
pool8 = MaxPooling2D(pool_size=(2, 2))(act8)

conv9 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool8)
batch9 = BatchNormalization()(conv9)
act9 = Activation("relu")(batch9)
pool9 = MaxPooling2D(pool_size=(2, 2))(act9)

conv10 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool9)
batch10 = BatchNormalization()(conv10)
act10 = Activation("relu")(batch10)

conv11 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act10)
batch11 = BatchNormalization()(conv11)
act11 = Activation("relu")(batch11)

up12 = UpSampling2D(size=(2, 2))(act11)
conv12 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up12)
batch12 = BatchNormalization()(conv12)
act12 = Activation("relu")(batch12)
concat12 = concatenate([act9,act12])

up13 = UpSampling2D(size=(2, 2))(concat12)
conv13 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up13)
batch13 = BatchNormalization()(conv13)
act13 =  Activation("relu")(batch13)
concat13 = concatenate([act8,act13])

up14 = UpSampling2D(size=(2, 2))(concat13)
conv14 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(up14)
batch14 = BatchNormalization()(conv14)
act14 = Activation("relu")(batch14)
concat14 = concatenate([act1,act14])

output_xnet = Conv2D(1, (1, 1), activation='sigmoid')(concat14)

model = Model(img_input, output_xnet)
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=[dice_coef_loss])
model.summary()

In [ ]:
callbacks = [ModelCheckpoint('Supression_lung_Xnet.h5', monitor='val_dice_coef_loss', verbose=1, save_best_only=True, mode='min')]
history = model.fit(X_train, y_train, batch_size = 64, epochs = 100, verbose = 1, validation_data = (X_test,y_test), callbacks = callbacks)

In [ ]:
model.save('Xnet_supression.h5')

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model = load_model('../input/package-models/Supression_lung_Xnet (1).h5', custom_objects=dependencies)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])
#results = loaded_model.evaluate(X_test,y_test,batch_size=64, verbose=1)
#print("Loss and dice_coef_loss:", results)

In [ ]:
number = 2
image_rx = np.squeeze(X_test[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)

supression_rx = np.squeeze(y_test[number].astype(np.float64))
plt.imsave('supression_000.jpg', supression_rx, cmap='gray')
supression_000 = cv2.imread('./supression_000.jpg',1)

predicted_rx = loaded_model.predict(X_test[[number]].astype(np.float64),verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
predicted_000 = cv2.imread('./predicted_000.jpg',1)



stack = np.hstack((image_000,supression_000,predicted_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

In [ ]:
length = len(SEGMENTATION_IMAGES)
images_seg = list()

In [ ]:
os.makedirs('./input/images')

In [ ]:
for number in range(0,length):
    predicted_rx = loaded_model.predict(SEGMENTATION_IMAGES[[number]].astype(np.float64),verbose = 1)
    predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
    plt.imsave('./input/images/'+seg[number]+'.png', predicted_rx, cmap = 'gray')

In [ ]:
number = 2
image_rx = np.squeeze(SEGMENTATION_IMAGES[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)


predicted_rx = loaded_model.predict(SEGMENTATION_IMAGES[[number]].astype(np.float64),verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
predicted_000 = cv2.imread('./predicted_000.jpg',1)



stack = np.hstack((image_000,predicted_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

# 6.1.1 - Gan for bone supression or segmentation from Chest X-Ray

In [ ]:
#def dice_loss(y_true, y_pred):
#    y_true = tf.dtypes.cast(y_true, tf.float32)
#    y_pred = tf.math.sigmoid(y_pred)
#    numerator = 2 * tf.reduce_sum(y_true * y_pred)
#    denominator = tf.reduce_sum(y_true + y_pred)
#    return 1 - (numerator / denominator)
beta = 0.5
def tversky_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = y_true * y_pred
    denominator = y_true * y_pred + beta * (1 - y_true) * y_pred + (1 - beta) * y_true * (1 - y_pred)
    return tf.subtract(1, tf.divide(tf.reduce_sum(numerator), tf.reduce_sum(denominator)))

def dice_coef_9cat(y_true, y_pred, smooth=1e-7):
    '''
    Dice coefficient for 10 categories. Ignores background pixel label 0
    Pass to model as metric during compile statement
    '''
    y_true_f = K.flatten(K.one_hot(K.cast(y_true, 'int32'), num_classes=10)[...,1:])
    y_pred_f = K.flatten(y_pred[...,1:])
    intersect = K.sum(y_true_f * y_pred_f, axis=-1)
    denom = K.sum(y_true_f + y_pred_f, axis=-1)
    return K.mean((2. * intersect / (denom + smooth)))

def dice_coef_9cat_loss(y_true, y_pred):
    '''
    Dice loss to minimize. Pass to model as loss during compile statement
    '''
    return 1 - dice_coef_9cat(y_true, y_pred)

In [ ]:
# Length of our images (1 channel)
print(len(supression_images), len(supression_targets))

In [ ]:
SUPRESSION_IMAGES_GAN = (np.array(supression_images).reshape((len(supression_images), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5
SUPRESSION_TARGETS_GAN = (np.array(supression_targets).reshape((len(supression_targets), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5

In [ ]:
SUPRESSION_IMAGES_GAN = (np.array(ima).reshape((len(ima), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5
SUPRESSION_IMAGES_OTSU_GAN = (np.array(ima_otsu).reshape((len(ima_otsu), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5
SUPRESSION_IMAGES_CANNY_GAN = (np.array(ima_canny).reshape((len(ima_canny), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5
SUPRESSION_TARGETS_GAN = (np.array(mas).reshape((len(mas), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5

In [ ]:
data_gen_args = dict(rotation_range=10,
                     width_shift_range=0.2,
                     height_shift_range=0.2)
datagen = ImageDataGenerator(**data_gen_args)

seed = 1

i = 0
images_aug = []
images_otsu_aug = []
images_canny_aug = []
masks_aug = []
for batch in tqdm(datagen.flow(SUPRESSION_IMAGES_GAN, SUPRESSION_TARGETS_GAN, batch_size=1,seed=seed)):
    images_aug.append(batch[0])
    i += 1
    if i > 703:
        break
i = 0        
for batch in tqdm(datagen.flow(SUPRESSION_IMAGES_OTSU_GAN, SUPRESSION_IMAGES_GAN, batch_size=1,seed=seed)):
    images_otsu_aug.append(batch[0])
    i += 1
    if i > 703:
        break
i = 0        
for batch in tqdm(datagen.flow(SUPRESSION_IMAGES_CANNY_GAN, SUPRESSION_IMAGES_OTSU_GAN, batch_size=1,seed=seed)):
    images_canny_aug.append(batch[0])
    i += 1
    if i > 703:
        break
i = 0
for batch in tqdm(datagen.flow(SUPRESSION_TARGETS_GAN, SUPRESSION_IMAGES_GAN, batch_size=1,seed=seed)):
    masks_aug.append(batch[0])
    i += 1
    if i > 703:
        break       

In [ ]:
len(images_aug)

In [ ]:
number = 400
aug_images0 = np.squeeze(images_aug[number].astype(np.float32)*127.5 + 127.5)
plt.imsave('image_aug_000.jpg', aug_images0, cmap='gray')
img0 = cv2.imread('./image_aug_000.jpg',1)

aug_images1 = np.squeeze(images_otsu_aug[number].astype(np.float32)*127.5 + 127.5)
plt.imsave('image_aug_001.jpg', aug_images1, cmap='gray')
img1 = cv2.imread('./image_aug_001.jpg',1)

aug_images2 = np.squeeze(images_canny_aug[number].astype(np.float32)*127.5 + 127.5)
plt.imsave('image_aug_002.jpg', aug_images2, cmap='gray')
img2 = cv2.imread('./image_aug_002.jpg',1)

aug_masks = np.squeeze(masks_aug[number].astype(np.float32)*127.5 + 127.5)
plt.imsave('mask_aug_000.jpg', aug_masks, cmap='gray')
mask0 = cv2.imread('./mask_aug_000.jpg',1)

stack = np.hstack((aug_images0, aug_images1, aug_images2, aug_masks))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

In [ ]:
#SUPRESSION_IMAGES_GAN_3C = list()
#SUPRESSION_TARGETS_GAN_3C = list()

#for img in SUPRESSION_IMAGES_GAN:
#    SUPRESSION_IMAGES_GAN_3C.append(np.concatenate((img,)*3, axis=-1))
#    
#for img in SUPRESSION_TARGETS_GAN:
#    SUPRESSION_TARGETS_GAN_3C.append(np.concatenate((img,)*3, axis=-1))
#
#SUPRESSION_IMAGES_GAN_3C = np.array(SUPRESSION_IMAGES_GAN_3C)
#SUPRESSION_TARGETS_GAN_3C = np.array(SUPRESSION_TARGETS_GAN_3C)

In [ ]:
buffer_size = len(SUPRESSION_IMAGES_GAN)
batch_size = 32

In [ ]:
SUPRESSION_IMAGES_GAN = np.append(SUPRESSION_IMAGES_GAN,images_aug)
SUPRESSION_IMAGES_OTSU_GAN = np.append(SUPRESSION_IMAGES_OTSU_GAN,images_otsu_aug)
SUPRESSION_IMAGES_CANNY_GAN = np.append(SUPRESSION_IMAGES_CANNY_GAN,images_canny_aug)
SUPRESSION_TARGETS_GAN = np.append(SUPRESSION_TARGETS_GAN,masks_aug)

In [ ]:
SUPRESSION_IMAGES_GAN = np.array(SUPRESSION_IMAGES_GAN).reshape((704*2, SIZE, SIZE, 1))
SUPRESSION_IMAGES_OTSU_GAN = np.array(SUPRESSION_IMAGES_OTSU_GAN).reshape((704*2, SIZE, SIZE, 1))
SUPRESSION_IMAGES_CANNY_GAN = np.array(SUPRESSION_IMAGES_CANNY_GAN).reshape((704*2, SIZE, SIZE, 1))
SUPRESSION_TARGETS_GAN = np.array(SUPRESSION_TARGETS_GAN).reshape((704*2, SIZE, SIZE, 1))

In [ ]:
len(SUPRESSION_IMAGES_GAN)

In [ ]:
number = 1000
aug_images0 = (np.squeeze(SUPRESSION_IMAGES_GAN[number].astype(np.float32))*127.5 + 127.5)
plt.imsave('image_aug_000.jpg', aug_images0, cmap='gray')
img0 = cv2.imread('./image_aug_000.jpg',1)

aug_images1 = (np.squeeze(SUPRESSION_IMAGES_OTSU_GAN[number].astype(np.float32))*127.5 + 127.5)
plt.imsave('image_aug_001.jpg', aug_images1, cmap='gray')
img1 = cv2.imread('./image_aug_001.jpg',1)

aug_images2 = (np.squeeze(SUPRESSION_IMAGES_CANNY_GAN[number].astype(np.float32))*127.5 + 127.5)
plt.imsave('image_aug_002.jpg', aug_images2, cmap='gray')
img2 = cv2.imread('./image_aug_002.jpg',1)

aug_masks = (np.squeeze(SUPRESSION_TARGETS_GAN[number].astype(np.float32))*127.5 + 127.5)
plt.imsave('mask_aug_000.jpg', aug_masks, cmap='gray')
mask0 = cv2.imread('./mask_aug_000.jpg',1)

stack = np.hstack((aug_images0, aug_images1, aug_images2, aug_masks))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

In [ ]:
train_dataset_X = tf.data.Dataset.from_tensor_slices(SUPRESSION_IMAGES_GAN).batch(batch_size)
train_dataset_otsu_X = tf.data.Dataset.from_tensor_slices(SUPRESSION_IMAGES_OTSU_GAN).batch(batch_size)
train_dataset_canny_X = tf.data.Dataset.from_tensor_slices(SUPRESSION_IMAGES_CANNY_GAN).batch(batch_size)
train_dataset_y = tf.data.Dataset.from_tensor_slices(SUPRESSION_TARGETS_GAN).batch(batch_size)

In [ ]:
"""
import tensorflow as tf  # TF 2.0


class SpectralNormalization(tf.keras.layers.Wrapper):
    def __init__(self, layer, iteration=1, eps=1e-12, training=True, **kwargs):
        self.iteration = iteration
        self.eps = eps
        self.do_power_iteration = training
        if not isinstance(layer, tf.keras.layers.Layer):
            raise ValueError(
                'Please initialize `TimeDistributed` layer with a '
                '`Layer` instance. You passed: {input}'.format(input=layer))
        super(SpectralNormalization, self).__init__(layer, **kwargs)

    def build(self, input_shape):
        self.layer.build(input_shape)

        self.w = self.layer.kernel
        self.w_shape = self.w.shape.as_list()

        self.v = self.add_weight(shape=(1, self.w_shape[0] * self.w_shape[1] * self.w_shape[2]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_v',
                                 dtype=tf.float32)

        self.u = self.add_weight(shape=(1, self.w_shape[-1]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_u',
                                 dtype=tf.float32)

        super(SpectralNormalization, self).build()

    def call(self, inputs):
        self.update_weights()
        output = self.layer(inputs)
        self.restore_weights()  # Restore weights because of this formula "W = W - alpha * W_SN`"
        return output
    
    def update_weights(self):
        w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])
        
        u_hat = self.u
        v_hat = self.v  # init v vector

        if self.do_power_iteration:
            for _ in range(self.iteration):
                v_ = tf.matmul(u_hat, tf.transpose(w_reshaped))
                v_hat = v_ / (tf.reduce_sum(v_**2)**0.5 + self.eps)

                u_ = tf.matmul(v_hat, w_reshaped)
                u_hat = u_ / (tf.reduce_sum(u_**2)**0.5 + self.eps)

        sigma = tf.matmul(tf.matmul(v_hat, w_reshaped), tf.transpose(u_hat))
        self.u.assign(u_hat)
        self.v.assign(v_hat)

        self.layer.kernel.assign(self.w / sigma)

    def restore_weights(self):
        self.layer.kernel.assign(self.w)
"""

In [ ]:
from keras.initializers import RandomNormal
def make_generator_model():
    
    class SpectralNormalization(tf.keras.layers.Wrapper):
        def __init__(self, layer, iteration=1, eps=1e-12, training=True, **kwargs):
            self.iteration = iteration
            self.eps = eps
            self.do_power_iteration = training
            if not isinstance(layer, tf.keras.layers.Layer):
                raise ValueError(
                    'Please initialize `TimeDistributed` layer with a '
                    '`Layer` instance. You passed: {input}'.format(input=layer))
            super(SpectralNormalization, self).__init__(layer, **kwargs)

        def build(self, input_shape):
            self.layer.build(input_shape)

            self.w = self.layer.kernel
            self.w_shape = self.w.shape.as_list()

            self.v = self.add_weight(shape=(1, self.w_shape[0] * self.w_shape[1] * self.w_shape[2]),
                                     initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                     trainable=False,
                                     name='sn_v',
                                     dtype=tf.float32)

            self.u = self.add_weight(shape=(1, self.w_shape[-1]),
                                     initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                     trainable=False,
                                     name='sn_u',
                                     dtype=tf.float32)

            super(SpectralNormalization, self).build()

        def call(self, inputs):
            self.update_weights()
            output = self.layer(inputs)
            self.restore_weights()  # Restore weights because of this formula "W = W - alpha * W_SN`"
            return output

        def update_weights(self):
            w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])

            u_hat = self.u
            v_hat = self.v  # init v vector

            if self.do_power_iteration:
                for _ in range(self.iteration):
                    v_ = tf.matmul(u_hat, tf.transpose(w_reshaped))
                    v_hat = v_ / (tf.reduce_sum(v_**2)**0.5 + self.eps)

                    u_ = tf.matmul(v_hat, w_reshaped)
                    u_hat = u_ / (tf.reduce_sum(u_**2)**0.5 + self.eps)

            sigma = tf.matmul(tf.matmul(v_hat, w_reshaped), tf.transpose(u_hat))
            self.u.assign(u_hat)
            self.v.assign(v_hat)

            self.layer.kernel.assign(self.w / sigma)

        def restore_weights(self):
            self.layer.kernel.assign(self.w)
    
    input_shape=(128,128,1)
    classes=1
    kernel_size = 3
    filter_depth = (64,128,256,512,0)
    init = RandomNormal(mean = 0.0, stddev = 0.02)

    img_input1 = Input(shape=input_shape)
    img_input2 = Input(shape=input_shape)
    img_input3 = Input(shape=input_shape)

    concat_input = tf.keras.layers.concatenate([img_input1,img_input2, img_input3])
    
    conv1 = SpectralNormalization(Conv2D(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat_input)
    batch1 = BatchNormalization()(conv1)
    act1 = layers.LeakyReLU(0.2)(batch1)

    conv2 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act1)
    batch2 = BatchNormalization()(conv2)
    act2 = layers.LeakyReLU(0.2)(batch2)

    conv3 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act2)
    batch3 = BatchNormalization()(conv3)
    act3 = layers.LeakyReLU(0.2)(batch3)

    conv4 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act3)
    batch4 = BatchNormalization()(conv4)
    act4 = layers.LeakyReLU(0.2)(batch4)

    conv5 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act4)
    batch5 = BatchNormalization()(conv5)
    act5 = layers.LeakyReLU(0.2)(batch5)

    conv6 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act5)
    batch6 = BatchNormalization()(conv6)
    act6 = layers.LeakyReLU(0.2)(batch6)
    concat6 = concatenate([act2,act6])

    conv7 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat6)
    batch7 = BatchNormalization()(conv7)
    act7 = layers.LeakyReLU(0.2)(batch7)
    concat7 = concatenate([act1,act7])

    conv8 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(concat7)
    batch8 = BatchNormalization()(conv8)
    act8 = layers.LeakyReLU(0.2)(batch8)

    conv9 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act8)
    batch9 = BatchNormalization()(conv9)
    act9 = layers.LeakyReLU(0.2)(batch9)

    conv10 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act9)
    batch10 = BatchNormalization()(conv10)
    act10 = layers.LeakyReLU(0.2)(batch10)

    conv11 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act10)
    batch11 = BatchNormalization()(conv11)
    act11 = layers.LeakyReLU(0.2)(batch11)

    conv12 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act11)
    batch12 = BatchNormalization()(conv12)
    act12 = layers.LeakyReLU(0.2)(batch12)
    concat12 = concatenate([act9,act12])

    conv13 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat12)
    batch13 = BatchNormalization()(conv13)
    act13 =  layers.LeakyReLU(0.2)(batch13)
    concat13 = concatenate([act8,act13])

    conv14 = SpectralNormalization(Conv2DTranspose(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat13)
    batch14 = BatchNormalization()(conv14)
    act14 = layers.LeakyReLU(0.2)(batch14)
    concat14 = concatenate([concat_input,act14])

    output_xnet = SpectralNormalization(Conv2D(1, (1, 1), activation='tanh'))(concat14)

    model = Model(inputs = [img_input1,img_input2,img_input3], outputs = output_xnet)
    
    model.summary()
        
    return model

In [ ]:
def make_generator_model():
    dependencies = {'dice_coef_loss': dice_coef_loss}
    model = load_model('../input/package-models/Supression_lung_Xnet (1).h5', custom_objects=dependencies)
    model.summary()

    return model

In [ ]:
from keras.initializers import RandomNormal
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)
    return g

def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g, training=True)
    if dropout:
        g = Dropout(0.5)(g, training=True)
    g = tf.keras.layers.concatenate([g, skip_in])
    g = Activation('relu')(g)
    return g

def make_generator_model():
    init = RandomNormal(stddev=0.02)
    input_shape=(256,256,1)
    
    img_input1 = Input(shape=input_shape)
    img_input2 = Input(shape=input_shape)
    img_input3 = Input(shape=input_shape)

    concat_input = tf.keras.layers.concatenate([img_input1,img_input2, img_input3])
    
    e1 = define_encoder_block(concat_input, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    d8 = decoder_block(d7, concat_input, 64, dropout=False)
    
    out_image = Conv2D(1, (1, 1), activation='tanh')(d8)
    
    model = Model(inputs = [img_input1,img_input2,img_input3], outputs = out_image)
    
    return model

In [ ]:
generator = make_generator_model()

In [ ]:
generator.summary()

In [ ]:
def make_discrimator_model():
    
    
    class SpectralNormalization(tf.keras.layers.Wrapper):
        def __init__(self, layer, iteration=1, eps=1e-12, training=True, **kwargs):
            self.iteration = iteration
            self.eps = eps
            self.do_power_iteration = training
            if not isinstance(layer, tf.keras.layers.Layer):
                raise ValueError(
                    'Please initialize `TimeDistributed` layer with a '
                    '`Layer` instance. You passed: {input}'.format(input=layer))
            super(SpectralNormalization, self).__init__(layer, **kwargs)

        def build(self, input_shape):
            self.layer.build(input_shape)

            self.w = self.layer.kernel
            self.w_shape = self.w.shape.as_list()

            self.v = self.add_weight(shape=(1, self.w_shape[0] * self.w_shape[1] * self.w_shape[2]),
                                     initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                     trainable=False,
                                     name='sn_v',
                                     dtype=tf.float32)

            self.u = self.add_weight(shape=(1, self.w_shape[-1]),
                                     initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                     trainable=False,
                                     name='sn_u',
                                     dtype=tf.float32)

            super(SpectralNormalization, self).build()

        def call(self, inputs):
            self.update_weights()
            output = self.layer(inputs)
            self.restore_weights()  # Restore weights because of this formula "W = W - alpha * W_SN`"
            return output

        def update_weights(self):
            w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])

            u_hat = self.u
            v_hat = self.v  # init v vector

            if self.do_power_iteration:
                for _ in range(self.iteration):
                    v_ = tf.matmul(u_hat, tf.transpose(w_reshaped))
                    v_hat = v_ / (tf.reduce_sum(v_**2)**0.5 + self.eps)

                    u_ = tf.matmul(v_hat, w_reshaped)
                    u_hat = u_ / (tf.reduce_sum(u_**2)**0.5 + self.eps)

            sigma = tf.matmul(tf.matmul(v_hat, w_reshaped), tf.transpose(u_hat))
            self.u.assign(u_hat)
            self.v.assign(v_hat)

            self.layer.kernel.assign(self.w / sigma)

        def restore_weights(self):
            self.layer.kernel.assign(self.w)
    
    init = RandomNormal(mean = 0.0, stddev = 0.02)
    inp = Input(shape=[SIZE, SIZE, 1], name='input_image')
    inp_otsu = Input(shape=[SIZE, SIZE, 1], name='input_image_otsu')
    inp_canny = Input(shape=[SIZE, SIZE, 1], name='input_image_canny')
    tar = Input(shape=[SIZE, SIZE, 1], name='target_image')
    
    x = tf.keras.layers.concatenate([inp,inp_otsu,inp_canny,tar])
    
    conv1 = SpectralNormalization(Conv2D(64, (5,5), strides = (2,2), padding='same', kernel_initializer = init))(x)
    batch1 = BatchNormalization()(conv1)
    act1 = layers.LeakyReLU(0.2)(batch1)
    drop1 = Dropout(0.3)(act1)
    
    conv2 = SpectralNormalization(Conv2D(128, (5,5), strides = (2,2), padding='same', kernel_initializer = init))(drop1)
    batch2 = BatchNormalization()(conv2)
    act2 = layers.LeakyReLU(0.2)(batch2)
    drop2 = Dropout(0.3)(act2)    
    
    flat = layers.Flatten()(drop2)
    last = layers.Dense(1, activation='sigmoid')(flat)
    
    return Model(inputs=[inp,inp_otsu,inp_canny, tar], outputs=last)   

In [ ]:
def make_discrimator_model():
    
    init = RandomNormal(mean = 0.0, stddev = 0.02)
    inp = Input(shape=[SIZE, SIZE, 1], name='input_image')
    inp_otsu = Input(shape=[SIZE, SIZE, 1], name='input_image_otsu')
    inp_canny = Input(shape=[SIZE, SIZE, 1], name='input_image_canny')
    tar = Input(shape=[SIZE, SIZE, 1], name='target_image')
    
    merged = tf.keras.layers.concatenate([inp,inp_otsu,inp_canny,tar])
                                    
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    flat = layers.Flatten()(d)
    patch_out = layers.Dense(1, activation='sigmoid')(flat) 
    model = Model(inputs=[inp,inp_otsu,inp_canny, tar], outputs=patch_out)
    
    return model

In [ ]:
discriminator = make_discrimator_model()

In [ ]:
discriminator.summary()

In [ ]:
import shutil

shutil.rmtree('./input/training_checkpoints')
shutil.rmtree('./input/images_gan')

In [ ]:
def dice_loss_2d(Y_gt, Y_pred):
    H, W, C = Y_gt.get_shape().as_list()[1:]
    smooth = 1e-5
    pred_flat = tf.reshape(Y_pred, [-1, H * W * C])
    true_flat = tf.reshape(Y_gt, [-1, H * W * C])
    pred_flat = tf.cast(pred_flat, dtype=tf.float32)
    true_flat = tf.cast(true_flat, dtype=tf.float32)
    intersection = 2.0 * tf.reduce_sum(pred_flat * true_flat, axis=1) + smooth
    denominator = tf.reduce_sum(pred_flat, axis=1) + tf.reduce_sum(true_flat, axis=1) + smooth
    loss = 1.0 - tf.reduce_mean(intersection / denominator)
    return loss

def generalised_dice_loss_2d(Y_gt, Y_pred):
    Y_gt = tf.cast(Y_gt, dtype=tf.float32)
    Y_pred = tf.cast(Y_pred, dtype=tf.float32)
    
    smooth = 1e-5
    w = tf.reduce_sum(Y_gt, axis=[1, 2])
    w = 1 / (w ** 2 + smooth)

    numerator = Y_gt * Y_pred
    numerator = w * tf.reduce_sum(numerator, axis=[1, 2])
    numerator = tf.reduce_sum(numerator, axis=1)

    denominator = Y_pred + Y_gt
    denominator = w * tf.reduce_sum(denominator, axis=[1, 2])
    denominator = tf.reduce_sum(denominator, axis=1)

    gen_dice_coef = 2 * numerator / (denominator + smooth)
    loss = tf.reduce_mean(1 - gen_dice_coef)
    return loss

In [ ]:
os.makedirs('./input/training_checkpoints')

MAE = tf.keras.losses.MeanAbsoluteError()
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = binary_cross_entropy(tf.multiply(tf.ones_like(disc_real_output),0.9), disc_real_output)
    fake_loss = binary_cross_entropy(tf.multiply(tf.zeros_like(disc_generated_output),0.1), disc_generated_output)
    total_disc_loss = real_loss + fake_loss
    return total_disc_loss

def generator_loss(disc_generated_output,fake_output_,real_output_):
    #gan_dice = generalised_dice_loss_2d(real_output_, fake_output_)
    #print(real_output_,fake_output_)
    gan_loss = binary_cross_entropy(tf.ones_like(disc_generated_output), disc_generated_output)
    l1_loss = MAE(real_output_,fake_output_)
    #gan_fl = fl(real_output_,fake_output_)
    total_gen_loss = gan_loss + (100 * l1_loss) #+ (60*gan_fl)
    return total_gen_loss, gan_loss

generator_optimizer = tf.keras.optimizers.Adam(1e-5, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

checkpoint_dir = './input/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
epochs = 3
num_examples_to_generate = 16
idx = np.random.randint(0, len(SUPRESSION_IMAGES_GAN),num_examples_to_generate)
seed1 = SUPRESSION_IMAGES_GAN[idx]
seed2 = SUPRESSION_IMAGES_OTSU_GAN[idx]
seed3 = SUPRESSION_IMAGES_CANNY_GAN[idx]

In [ ]:
import datetime
log_dir="../logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
def train_steps(input_image,input_image_otsu,input_image_canny,target, epoch):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator([input_image,input_image_otsu,input_image_canny], training=True)
        real_output = discriminator([input_image,input_image_otsu,input_image_canny, target], training=True)
        fake_output = discriminator([input_image,input_image_otsu,input_image_canny, generated_images], training=True)
        gen_total_loss, gen_gan_loss = generator_loss(fake_output, generated_images, target)
        disc_loss = discriminator_loss(real_output, fake_output)
    gradients_of_generator = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    with summary_writer.as_default():
        tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
        tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
        #tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
        tf.summary.scalar('disc_loss', disc_loss, step=epoch)
    return gen_total_loss,disc_loss

In [ ]:
os.makedirs('./input/images_gan')
def generate_and_save_images(model, epoch, test_input,test_input2,test_input3):
    predictions = model([test_input,test_input2,test_input3],training = False)

    fig = plt.figure(figsize=(8,8))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    plt.savefig('./input/images_gan/image_at_epoch_{:04d}.png'.format(epoch))   
    plt.show()

In [ ]:
def plot_loss(epoch, g_losses, d_losses):
    plt.figure(figsize=(10,5))
    plt.title("Loss, Epochs 0-" + str(epoch))
    plt.plot(g_losses,label="Generator")
    plt.plot(d_losses,label="Discriminator")
    #plt.plot(gan_dice_loss,label="Dice Loss Generator")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [ ]:
import numpy

def train(X_dataset,X_train_dataset_otsu,X_train_dataset_canny,y_dataset, epochs):
    g_loss = []
    d_loss = []
    gan_dice_loss = []
    #gan_dice_ = []
    for epoch in range(epochs):
        start = time.time()
        #print(start,X_dataset, y_dataset)
        for image_batch,image_batch_otsu,image_batch_canny,target_batch in zip(X_dataset,X_train_dataset_otsu,X_train_dataset_canny,y_dataset):
            generator_loss, discriminator_loss = train_steps(image_batch,image_batch_otsu,image_batch_canny,target_batch, epochs)
        
        g_loss.append(generator_loss.numpy())
    
        d_loss.append(discriminator_loss.numpy())
        
        #gan_dice_loss.append(gan_dice.numpy())
        
        display.clear_output(wait = True)
        print("Generator loss: " + str(g_loss))
        print("Discriminator loss: " + str(d_loss))
        #print("Dice Loss:" + str(gan_dice_loss))
        plot_loss(epoch, g_loss, d_loss)
        generate_and_save_images(generator, epoch + 1, seed1,seed2,seed3)

        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time() - start))

In [ ]:
def display_image(epoch_no):
    return PIL.Image.open('./input/images_gan/image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
train(train_dataset_X,train_dataset_otsu_X,train_dataset_canny_X,train_dataset_y, epochs)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
display_image(epochs)

*GIF*

In [ ]:
import glob
import imageio

anim_file = 'gan_segmentation.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('./input/images_gan/image*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

In [ ]:
pip install git+https://github.com/tensorflow/docs

In [ ]:
import tensorflow_docs.vis.embed as embed

embed.embed_file(anim_file)

In [ ]:
generator.save('generator_model.h5')
discriminator.save('discriminartor_model.h5')

In [ ]:
generator.load_weights('./generator_model.h5')

In [ ]:
from keras.initializers import RandomNormal
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)
    return g

def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g, training=True)
    if dropout:
        g = Dropout(0.5)(g, training=True)
    g = tf.keras.layers.concatenate([g, skip_in])
    g = Activation('relu')(g)
    return g


init = RandomNormal(stddev=0.02)
input_shape=(256,256,1)

img_input1 = Input(shape=input_shape)
img_input2 = Input(shape=input_shape)
img_input3 = Input(shape=input_shape)

concat_input = tf.keras.layers.concatenate([img_input1,img_input2, img_input3])

e1 = define_encoder_block(concat_input, 64, batchnorm=False)
e2 = define_encoder_block(e1, 128)
e3 = define_encoder_block(e2, 256)
e4 = define_encoder_block(e3, 512)
e5 = define_encoder_block(e4, 512)
e6 = define_encoder_block(e5, 512)
e7 = define_encoder_block(e6, 512)
b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
b = Activation('relu')(b)
d1 = decoder_block(b, e7, 512)
d2 = decoder_block(d1, e6, 512)
d3 = decoder_block(d2, e5, 512)
d4 = decoder_block(d3, e4, 512, dropout=False)
d5 = decoder_block(d4, e3, 256, dropout=False)
d6 = decoder_block(d5, e2, 128, dropout=False)
d7 = decoder_block(d6, e1, 64, dropout=False)
d8 = decoder_block(d7, concat_input, 64, dropout=False)

out_image = Conv2D(1, (1, 1), activation='tanh')(d8)

model = Model(inputs = [img_input1,img_input2,img_input3], outputs = out_image)
opt = Adam(1e-5, beta_1=0.5)
model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
return model

In [ ]:
opt = Adam(1e-5, beta_1=0.5)
generator.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])

In [ ]:
loaded_model = load_model('./generator_model.h5')
loaded_model.compile(loss=['mae'], optimizer=opt)
results = loaded_model.evaluate([SUPRESSION_IMAGES_GAN,SUPRESSION_IMAGES_OTSU_GAN,SUPRESSION_IMAGES_CANNY_GAN], SUPRESSION_TARGETS_GAN,batch_size=32, verbose=1)
print("Loss:", results)

In [ ]:
results = loaded_model.evaluate([SUPRESSION_IMAGES_GAN,SUPRESSION_IMAGES_OTSU_GAN,SUPRESSION_IMAGES_CANNY_GAN], SUPRESSION_TARGETS_GAN, verbose=1)
print("Loss:", results)

Masks

In [ ]:
X_train = (np.array(ima).reshape((len(ima), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5
X_train_otsu = (np.array(ima_otsu).reshape((len(ima_otsu), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5
X_train_canny = (np.array(ima_canny).reshape((len(ima_canny), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5
y_train = (np.array(mas).reshape((len(mas), SIZE, SIZE, 1)).astype('float64') - 127.5) / 127.5

In [ ]:
number = 300
image_rx = np.squeeze(X_train[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)

#supression_rx = np.squeeze(y_train[number].astype(np.float64))
#plt.imsave('supression_000.jpg', supression_rx, cmap='gray')
#supression_000 = cv2.imread('./supression_000.jpg',1)

predicted_rx = generator.predict([X_train[[number]].astype(np.float64),X_train_otsu[[number]].astype(np.float64),X_train_canny[[number]].astype(np.float64)],verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
img = cv2.imread('./predicted_000.jpg',1)

img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((5,5),np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

cnt0 = contours[0].reshape(-1,2)
epsilon0 = 0.000001*cv2.arcLength(cnt0,True)
approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
try: 
    cnt1 = contours[1].reshape(-1,2)
    epsilon1 = 0.0000001*cv2.arcLength(cnt1,True)
    approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
except:
    pass
canvas = np.zeros(thresh.shape, np.uint8)

#draw = cv2.drawContours(canvas, approx, -1, (0,255,0), 3)
#plt.imsave('predicted_000.jpg', thresh, cmap='gray')

cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
try:
    cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
except:
    pass
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imwrite('predicted_000.png',thresh)
predicted_000 = cv2.imread('./predicted_000.png',1)

out = np.zeros_like(thresh)
for i in range(0,SIZE):
    for j in range(0,SIZE):
        #out[thresh == 255] = image_rx[thresh == 255]
        if thresh[i,j] == 255:
            out[i,j] = image_rx[i,j]*127.5 + 127.5
        
cv2.imwrite('out_000.png',out)
out_000 = cv2.imread('./out_000.png',1)

stack = np.hstack((image_000, predicted_000,out_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

COVID

In [ ]:
X_train = (np.array(covid_images).reshape((len(covid_images), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5
X_train_otsu = (np.array(covid_images_otsu).reshape((len(covid_images_otsu), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5
X_train_canny = (np.array(covid_images_canny).reshape((len(covid_images_canny), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5

In [ ]:
length = len(covid_image_name)
#os.makedirs('./input/images_segment')

for number in range(0,length):
    image_rx = np.squeeze(X_train[number].astype(np.float64))
    predicted_rx = generator.predict([X_train[[number]].astype(np.float64),X_train_otsu[[number]].astype(np.float64),X_train_canny[[number]].astype(np.float64)],verbose = 1)
    predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
    
    plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
    img = cv2.imread('./predicted_000.jpg',1)

    img_grey = img[:,:,0]
    min_ = img_grey.min()
    max_ = img_grey.max()
    ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

    cnt0 = contours[0].reshape(-1,2)
    epsilon0 = 0.000001*cv2.arcLength(cnt0,True)
    approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)

    try: 
        cnt1 = contours[1].reshape(-1,2)
        epsilon1 = 0.0000001*cv2.arcLength(cnt1,True)
        approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
    except:
        pass

    canvas = np.zeros(thresh.shape, np.uint8)

    cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
    try:
        cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
    except:
        pass
    ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cv2.imwrite('predicted_000.png',thresh)
    predicted_000 = cv2.imread('./predicted_000.png',1)

    out = np.zeros_like(thresh)
    for i in range(0,128):
        for j in range(0,128):
            if thresh[i,j] == 255:
                out[i,j] = image_rx[i,j]*127.5 + 127.5
                
    cv2.imwrite('./input/images_segment/'+covid_image_name[number],out)

In [ ]:
length

In [ ]:
covid = "./input/images_segment"
covid_list = os.listdir(covid)
len(covid_list)

In [ ]:
length = len(covid_image_name)
os.makedirs('./input/segment')

for number in range(0,length):
    image_rx = np.squeeze(X_train[number].astype(np.float64))
    predicted_rx = generator.predict(X_train[[number]].astype(np.float64),verbose = 1)
    predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
    
    #plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
    #img = cv2.imread('./predicted_000.jpg',1)

    #img_grey = img[:,:,0]
    #min_ = img_grey.min()
    #max_ = img_grey.max()
    #ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

    #cnt0 = contours[0].reshape(-1,2)
    #epsilon0 = 0.000001*cv2.arcLength(cnt0,True)
    #approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)

    #cnt1 = contours[1].reshape(-1,2)
    #epsilon1 = 0.0000001*cv2.arcLength(cnt1,True)
    #approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)

    #canvas = np.zeros(thresh.shape, np.uint8)

    #cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
    #cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
    #ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #cv2.imwrite('predicted_000.png',thresh)
    #predicted_000 = cv2.imread('./predicted_000.png',1)

    #out = np.zeros_like(thresh)
    #for i in range(0,128):
    #    for j in range(0,128):
    #        if thresh[i,j] == 255:
    #            out[i,j] = image_rx[i,j]*127.5 + 127.5
                
    cv2.imwrite('./input/segment/'+covid_image_name[number],predicted_rx)

# 6.2 - Read and transform images

In [ ]:
seg_path = os.path.join("./input/images/")
#segm = os.listdir(seg_path)
IMAGES_SEG = list()
for i in tqdm(names):
    path_images = seg_path + str(i)
    image = cv2.imread(path_images,cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    IMAGES_SEG.append(image)

In [ ]:
IMAGES_SEG = np.array(IMAGES_SEG)
IMAGES_SEG = IMAGES_SEG.reshape((len(IMAGES_SEG), SIZE, SIZE, 1))

In [ ]:
IMAGES_SEG_3C = list()

for img in IMAGES_SEG:
    img_3C = np.concatenate((img,)*3, axis=-1)
    IMAGES_SEG_3C.append(img_3C)
    
IMAGES_SEG_3C = np.array(IMAGES_SEG_3C)
IMAGES_SEG_3C = IMAGES_SEG_3C.astype('float32') / 255

# 7. Lung segmentation from Chest X-Ray

In [ ]:
number = 3
image_rx = np.squeeze(SEGMENTATION_IMAGES[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)

supression_rx = np.squeeze(IMAGES_SEG_3C[number].astype(np.float64))
plt.imsave('supression_000.jpg', supression_rx, cmap='gray')
supression_000 = cv2.imread('./supression_000.jpg',1)

supression1_rx = np.squeeze(SEGMENTATION_MASKS[number].astype(np.float64))
plt.imsave('supression1_000.jpg', supression1_rx, cmap='gray')
supression1_000 = cv2.imread('./supression1_000.jpg',1)


stack = np.hstack((image_000,supression_000, supression1_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

In [ ]:
n_classes = 1
activation = 'sigmoid'
LR = 0.0001
optim = keras.optimizers.Adam(LR)
BATCH_SIZE = 16
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(IMAGES_SEG_3C, SEGMENTATION_MASKS, test_size=0.30, random_state=12)

x_train = preprocess_input1(X_train_s)
x_val = preprocess_input1(X_test_s)

model = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes = n_classes, activation = activation)
model.compile('Adam', loss='binary_crossentropy', metrics=[dice_coef_loss])
print(model.summary())

In [ ]:
callbacks = [ModelCheckpoint('Segmenter_lung.h5', monitor='val_dice_coef_loss', verbose=1, save_best_only=True, mode='min')]
history = model.fit(x_train, y_train_s, batch_size = 16, epochs = 300, verbose = 1, validation_data = (x_val,y_test_s), callbacks = callbacks)

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model = load_model('./Segmenter_lung.h5', custom_objects=dependencies)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])

In [ ]:
number = 200
image_rx = np.squeeze(X_test_s[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)

supression_rx = np.squeeze(y_test_s[number].astype(np.float64))
plt.imsave('supression_000.jpg', supression_rx, cmap='gray')
supression_000 = cv2.imread('./supression_000.jpg',1)

predicted_rx = loaded_model.predict(X_test_s[[number]].astype(np.float64),verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
predicted_000 = cv2.imread('./predicted_000.jpg',1)



stack = np.hstack((image_000,supression_000,predicted_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

# 8. Classification Normal x Pneumonia

In [ ]:
summary = pd.read_csv("../input/coronahack-chest-xraydataset/Chest_xray_Corona_dataset_Summary.csv")
summary

In [ ]:
metadata = pd.read_csv("../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv")
metadata_ = metadata['Label']
metadata_

In [ ]:
metadata

In [ ]:
covid_image_name

In [ ]:
le = preprocessing.LabelEncoder()
target = le.fit_transform(metadata_)
target

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model = load_model('../input/package-models/Supression_lung_Xnet (1).h5', custom_objects=dependencies)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])

In [ ]:
os.makedirs('./input/images_covid')
length = len(covid_image_name)

for number in range(0,length):
    predicted_rx = loaded_model.predict(COVID_IMAGES[[number]].astype(np.float64),verbose = 1)
    predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
    plt.imsave('./input/images_covid/'+covid_image_name[number], predicted_rx, cmap = 'gray')

In [ ]:
number = 28
image_rx = np.squeeze(COVID_IMAGES[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)


predicted_rx = loaded_model.predict(COVID_IMAGES[[number]].astype(np.float64),verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
predicted_000 = cv2.imread('./predicted_000.jpg',1)


stack = np.hstack((image_000,predicted_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(8,24))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

In [ ]:
covid_images_for_seg = list()

for i in tqdm(covid_image_name):
    path_images_train = './input/images_covid/' + str(i)
    image = cv2.imread(path_images_train,cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    covid_images_for_seg.append(image)


COVID_IMAGES_4SEG = np.array(covid_images_for_seg)

COVID_IMAGES_4SEG = COVID_IMAGES_4SEG.reshape((len(COVID_IMAGES_4SEG), SIZE, SIZE, 1))

COVID_IMAGES_4SEG = COVID_IMAGES_4SEG.astype('float32') / 255    

In [ ]:
COVID_IMAGES_4SEG_3C = list()

for img in COVID_IMAGES_4SEG:
    img_3C = np.concatenate((img,)*3, axis=-1)
    COVID_IMAGES_4SEG_3C.append(img_3C)
    
COVID_IMAGES_4SEG_3C = np.array(COVID_IMAGES_4SEG_3C)

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model = load_model('../input/package-model-2/Segmenter_lung (5).h5', custom_objects=dependencies)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])

In [ ]:
os.makedirs('./input/images_covid_segmentation')
length = len(covid_image_name)

for number in range(0,length):
    predicted_rx = loaded_model.predict(COVID_IMAGES_4SEG_3C[[number]].astype(np.float64),verbose = 1)
    predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
    plt.imsave('./input/images_covid_segmentation/'+covid_image_name[number], predicted_rx, cmap = 'gray')

In [ ]:
number = 28


image_rx = np.squeeze(COVID_IMAGES_4SEG_3C[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)


predicted_rx = loaded_model.predict(COVID_IMAGES_4SEG_3C[[number]].astype(np.float64),verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')

img = cv2.imread('./predicted_000.jpg',1)
img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((5,5),np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
plt.imsave('predicted_000.jpg', thresh, cmap='gray')
predicted_000 = cv2.imread('./predicted_000.jpg',1)

stack = np.hstack((image_000,predicted_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(8,24))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()